<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_ex06notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex06notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




板書や口頭で補足する前提なので，この notebook だけでは説明が不完全です．

今回の話は，学部の科目「機械学習I」でも出てきています（復習しつつ一歩先へ進む感じ）．受講していないひとは，以下をどうぞ．

- 2024年度「機械学習I」 第5回 https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024#ex05
- 2024年度「機械学習I」 第6回 https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2024#ex06



----
## 準備
----


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc  # アニメーションのため
import pandas as pd
import seaborn
seaborn.set()

# scikit-learn のいろいろ
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_moons
from sklearn.neural_network import MLPRegressor, MLPClassifier

2次元2クラス識別のサンプルデータ

In [ ]:
moonX, moonY = make_moons(n_samples=200, noise=0.25)

# グラフを描く
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(moonX[moonY==0, 0], moonX[moonY==0, 1])
ax.scatter(moonX[moonY==1, 0], moonX[moonY==1, 1])
ax.set_xlim(-1.5, 2.5)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
plt.show()

---
## 階層型ニューラルネットワーク
---


---
### 線形モデルの限界

#### 線形回帰

$D$次元入力の線形回帰モデル

$$
f(\mathbf{x}) = w_0 + w_1x_1 + w_2x_2 + \cdots + w_Dx_D
$$

は，$\mathbf{x}$ と出力の正解が作る $(D+1)$ 次元空間の中で $D$次元の平面（超平面）を表す．
そのため，平面にうまく当てはまらないようなデータではよい予測ができない．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/planefitting2.png" width="75%">

#### ロジスティック回帰

$D$ 次元入力を 2 クラスに分類するロジスティック回帰モデル

$$
\begin{aligned}
f(\mathbf{x}) &= \sigma(w_0 + w_1x_1+\cdots + w_Dx_D) = \sigma\left(w_0 + \sum_{d=1}^{D}w_dx_d \right) \\
&= \frac{1}{1+\exp{\left( - \left( w_0 + \sum_{d=1}^{D}w_dx_d \right) \right)}}
\end{aligned}
$$

において，$f(\mathbf{x}) = \frac{1}{2} \Leftrightarrow w_0 + \sum_{d=1}^{D} w_dx_d = 0$ だから，2クラスの識別境界は平面（$D$次元空間中の $(D-1)$次元超平面，$D=2$のときは直線）となる．
多クラスの場合も同様．そのため，上記の例のような場合にはうまく識別できない．

上記の2次元2クラス識別の問題をロジスティック回帰で解いてみると...

In [ ]:
# ロジスティック回帰
model = LogisticRegression()
model.fit(moonX, moonY)
yy = model.predict(moonX)
cc = np.sum(yy == moonY)
print(f'正答率: {cc}/{len(yy)} = {cc/len(yy):.3f}')

識別境界を可視化してみると...

In [ ]:
# グラフ描画用のグリッドデータの作成
K = 2
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
npoints = 100
dx, dy = (xmax - xmin)/npoints, (ymax - ymin)/npoints
x_mesh, y_mesh = np.mgrid[xmin:xmax:dx, ymin:ymax:dy]
X_mesh = np.dstack((x_mesh, y_mesh))

# X_mesh の各点における事後確率の推定
p = model.predict_proba(X_mesh.reshape((-1, 2)))
pp = p.reshape((X_mesh.shape[0], X_mesh.shape[1], K))

# グラフの描画
fig, ax = plt.subplots()
cmap = ['Blues', 'Oranges', 'Greens']
for k in range(K):
    ax.scatter(moonX[moonY==k, 0], moonX[moonY==k, 1])
    ax.contourf(x_mesh, y_mesh, pp[:, :, k], levels=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
plt.show()

----
### 階層型ニューラルネットワーク

#### 例: 2層ニューラルネットによる非線形回帰

##### モデルの構造と活性化関数

$D$ 次元入力 $\mathbf{x}$ から $y$ の値を予測する回帰モデル $f(\mathbf{x})$ を，次式のように定義してみる．

$$
\begin{aligned}
f(x) = z^{\rm O} &= \sum_{h=1}^{H} w_h^{\rm O} z_h^{\rm H} \\
z_h^{\rm H} &= \sigma\left( b_{h}^{\rm H} + \sum_{d=1}^{D}w_{h, d}^{\rm H} x_d  \right)
\end{aligned}
$$
$\sigma(s)$ は **活性化関数** (activation function) と呼ばれるものである（どのようなものかは後述）．
このモデルのパラメータは，$w_h^{\rm O}, b_h^{\rm H}$ および $w_{h, d}^{H}$ ($h = 1, 2, \ldots, H, d = 1, 2, \ldots , D$) である．
このモデルで適当な入力 $\mathbf{x}$ に対する出力 $z^{\rm O}$ を計算する場合，$\mathbf{x}$ を使って $z_h^{\rm H}$ を求めるステップと，$z_h^{\rm H}$ を使って $z^{\rm O}$ の値を求めるステップの2段階に分けることができる → 階層構造，ニューロン．
このモデルの場合は隠れ層 (hidden layer) と出力層の2層から成る．





隠れ層のニューロンが非線形の活性化関数をもつことで，$f(x)$ は非線形関数を表すことが可能となる．ニューラルネットでよく用いられるのは，ロジスティック回帰モデルでも使われている**シグモイド関数**(sigmoid function)の他，**双曲線正接関数**(hyperbolic tangent function)，**Rectified Linear 関数**（**ReLU関数**ともいいます）など（注）．

$$
\begin{aligned}
\mbox{Logistic Sigmoid}\quad  & \sigma(s) = \frac{1}{1+{\rm e}^{-s}}\\
\mbox{Hyperbolic Tangent}\quad & \sigma(s) = {\rm tanh}(s) = \frac{{\rm e}^{s} - {\rm e}^{-s}}{{\rm e}^{s} + {\rm e}^{-s}}\\
\mbox{Rectified Linear}\quad & \sigma(s) = \left\{
    \begin{array}{ll}
    s & (s \ge 0)\\
    0 & {\rm otherwise}\\
\end{array} \right.
\end{aligned}
$$

<span style="font-size: 75%">
※注: 「シグモイド」は「S字状の」という意味の語なので，双曲線正接関数なども含めて類似した形をしている関数たちの総称です．式(X)のものは，正確には「ロジスティックシグモイド関数」(logistic sigmoid function)といいます．
また，「Rectified Linear」な活性化関数を採用したニューロンを Rectified Linear Unit と呼ぶことから，この活性化関数を「ReLU」と呼ぶことがあります．
</span>


In [ ]:
# 活性化関数の値を計算
xmin, xmax = -4, 4
X = np.linspace(xmin, xmax, num=100)
Y1 = 1/(1+np.exp(-X)) # ロジスティックシグモイド
Y2 = np.tanh(X)       # 双曲線正接
Y3 = np.maximum(X, 0) # ReLU

# グラフに描く
fig = plt.figure(figsize=(12,3))
ax1 = fig.add_subplot(131)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(-1.2, 1.2)
ax1.axhline(y=0, color='black', linestyle='-')
ax1.axvline(x=0, color='black', linestyle='-')
ax1.axhline(y=1, color='gray', linestyle='--')
ax1.plot(X, Y1, c='red', linewidth=3, label='Sigmoid')
ax1.legend()
ax2 = fig.add_subplot(132)
ax2.set_xlim(xmin, xmax)
ax2.set_ylim(-1.2, 1.2)
ax2.axhline(y=0, color='black', linestyle='-')
ax2.axvline(x=0, color='black', linestyle='-')
ax2.axhline(y=1, color='gray', linestyle='--')
ax2.axhline(y=-1, color='gray', linestyle='--')
ax2.plot(X, Y2, c='red', linewidth=3, label='tanh(s)')
ax2.legend()
ax3 = fig.add_subplot(133)
ax3.set_xlim(xmin, xmax)
ax3.set_ylim(xmin, xmax)
ax3.axhline(y=0, color='black', linestyle='-')
ax3.axvline(x=0, color='black', linestyle='-')
ax3.plot(X, Y3, c='red', linewidth=3, label='ReLU')
ax3.legend()
plt.show()

##### 学習

学習データが $(\mathbf{x}_n, y_n)$ ($n=1,2,\ldots, N$) と与えられたときに，線形回帰と同様に，二乗誤差

$$
E = \frac{1}{2}\sum_{n=1}^{N} (y_n - f(\mathbf{x}_n))^2
$$

が最小となるようなパラメータを求める．ただし，線形回帰と違って一撃の計算では最適なパラメータが求まらないので，勾配法を用いる．そのため，活性化関数は微分可能でなければならない．

入力が1次元（$D=1$）の場合に，パラメータの勾配を求めてみよう．
$e_n = \frac{1}{2}(y_n - z_n^{\rm O})^2$ とおく．
活性化関数にロジスティックシグモイドを用いる場合，$\frac{d \sigma(s)}{ds} = \sigma(s)(1-\sigma(s))$ となる．
これを用いて，
$\frac{\partial e_n}{\partial w_h^{\rm O}}$, $\frac{\partial e_n}{\partial w_h^{\rm H}}$, $\frac{\partial e_n}{\partial b_h^{\rm H}}$ を求めてみよう．

ニューラルネットの出力の計算時には，ニューラルネットへ入力された信号は，入力に近い側の層から順に層の間を伝わって出力へと至る．
これに対して，勾配の計算時には，出力層で計算された誤差の値が，入力側の層へと逆向きに伝わっていくとみなすことができる．
そのため，勾配法によるニューラルネットの学習は，**誤差逆伝播学習** (error back-propagation learning)と呼ばれることもある．

#### 例: 2層ニューラルネットによる2クラス識別

次式は，$D$次元入力を2クラスに分類する2層ニューラルネットモデルである．

$$
\begin{aligned}
f(\mathbf{x}) = z^{\rm O} &= \sigma^{\rm O} \left( b^{\rm O} + \sum_{h=1}^{H} w_h^{\rm O} z_h^{\rm H} \right) \\
z_h^{\rm H} &= \sigma^{\rm H}\left( b_{h}^{\rm H} + \sum_{d=1}^{D}w_{h,d}^{\rm H} x_d  \right)
\end{aligned}
$$

隠れ層の活性化関数 $\sigma^{\rm H}$ には，ロジスティックシグモイドや ReLU 等を用いる．
出力層の活性化関数 $\sigma^{\rm O}$ には，ロジスティック回帰と同様にロジスティックシグモイド関数を用いる．

出力クラスの正解を付した学習データに対して，ロジスティック回帰と同様に，交差エントロピーを最小化するようなパラメータを勾配法によって求めることで，2クラス識別を学習させることができる．

In [ ]:
#@title デモ: 学習させるモデルを選んでね
option = 'logistic' #@param ['logistic', '2-layer (1000,)', '2-layer (2000,)', '3-layer (100, 100)', '3-layer (1000, 1000)'] {allow-input: false}

if option == 'logistic':
    model = LogisticRegression()
else:
    if option == '2-layer (1000,)':
        numNeurons = (1000, )
    elif option == '2-layer (2000,)':
        numNeurons = (2000, )
    elif option == '3-layer (100, 100)':
        numNeurons = (100, 100, )
    elif option == '3-layer (1000, 1000)':
        numNeurons = (1000, 1000, )
    model = MLPClassifier(hidden_layer_sizes=numNeurons, activation='relu', verbose=True)

model.fit(moonX, moonY)
yy = model.predict(moonX)
cc = np.sum(yy == moonY)
print()
print(f'{option}   正答率: {cc}/{len(yy)} = {cc/len(yy):.3f}')

In [ ]:
# グラフ描画用のグリッドデータの作成
K = 2
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
npoints = 100
dx, dy = (xmax - xmin)/npoints, (ymax - ymin)/npoints
x_mesh, y_mesh = np.mgrid[xmin:xmax:dx, ymin:ymax:dy]
X_mesh = np.dstack((x_mesh, y_mesh))

# X_mesh の各点における事後確率の推定
p = model.predict_proba(X_mesh.reshape((-1, 2)))
pp = p.reshape((X_mesh.shape[0], X_mesh.shape[1], K))

# グラフの描画
fig, ax = plt.subplots()
cmap = ['Blues', 'Oranges', 'Greens']
for k in range(K):
    ax.scatter(moonX[moonY==k, 0], moonX[moonY==k, 1])
    ax.contourf(x_mesh, y_mesh, pp[:, :, k], levels=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
plt.show()

#### 階層型ニューラルネットワークと深層学習

出力ニューロンは複数にすることもできる．例えば，$K$ 個出力ニューロンを持つニューラルネットで，出力層の活性化関数を softmax として交差エントロピーを最小化するように学習させれば，$K$ クラスの識別ができる．
また，隠れ層を複数用いることもできる．

隠れ層の数や層ごとのニューロンの数，活性化関数の選択，etc. はハイパーパラメータ



**深層学習** (deep learning)という語の意味は...